In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
cd /content/drive/MyDrive/NLP_Project

/content/drive/MyDrive/NLP_Project


In [50]:
ls

GoogleNews-vectors-negative300.bin  words_147mb.txt   words_569.5mb.txt
NLP_PRO.ipynb                       words_1.73gb.txt  words_62mb.txt
vocabulary.pkl                      words_2.02gb.txt  words_870mb.txt
webbase_all/                        words_2.32gb.txt  words_91mb.txt
words_1.45gb.txt                    words_300mb.txt


In [51]:
# !tar zxvf umbc_webbase_corpus.tgz

# Import required libraries

In [52]:
import os

In [53]:
# dataset_folder_path = os.path.join(os.getcwd(), 'webbase_all')

In [54]:
# count = 0
# txt_count = 0
# txt_files = []

# pos_tagged_count = 0

# # Iterate directory
# for path in os.listdir(dataset_folder_path):
    
#     file_path = os.path.join(dataset_folder_path, path)
    
#     # check if current path is a file
#     if os.path.isfile(file_path):
#         count += 1

#     if file_path.endswith('.txt'):
#         txt_count += 1
#         txt_files.append(file_path)
    
#     else:
#         os.remove(file_path)
    
# print("Number of files: ", count)
# print("Number of text files: ", txt_count)
# print("Number of pos tagged files: ", pos_tagged_count)

In [55]:
# # import the necessary libraries
# import nltk
# import string
# import re

# from nltk.tokenize import word_tokenize

# import nltk
# from nltk.corpus import stopwords

# nltk.download('punkt')
# nltk.download('stopwords')

# stop_words = set(stopwords.words('english'))

# **Preprocessing sentences**

In [56]:
# def preprocess(sentence):
#     # convert to lowercase
#     sentence = sentence.lower()

#     # remove numbers
#     sentence = re.sub(r'\d+', '', sentence)

#     # remove punctuations
#     # anything that isn’t an alphanumeric character or whitespace is replaced with a blank string
#     sentence = re.sub(r'[^\w\s]', '', sentence)

#     # remove stopwords
#     tokens = word_tokenize(sentence)
#     result = [i for i in tokens if not i in stop_words]
#     sentence = (" ").join(result)

#      # remove whitespace
#     sentence = sentence.strip()

#     return sentence 

In [57]:
# output_file = open("words.txt", "w")
# result_lines = []

# for file_path in txt_files[:2]:
#     f = open(file_path, 'r')

#     for line in f.readlines():
#         result = preprocess(line)

#         if (result != ""):
#             result_lines.append(result+"\n")

# output_file.writelines(result_lines)
# output_file.close()

# Load the dataset file

In [58]:
ls

GoogleNews-vectors-negative300.bin  words_147mb.txt   words_569.5mb.txt
NLP_PRO.ipynb                       words_1.73gb.txt  words_62mb.txt
vocabulary.pkl                      words_2.02gb.txt  words_870mb.txt
webbase_all/                        words_2.32gb.txt  words_91mb.txt
words_1.45gb.txt                    words_300mb.txt


In [59]:
words_dataset_path = "words_62mb.txt"
# words_dataset_path = "words_300mb.txt"

In [60]:
words_dataset_file = open(words_dataset_path, "r")

sentences = words_dataset_file.readlines()

words_dataset_file.close()

In [61]:
import gensim
import pickle

# Load pre-trained word embeddings (e.g. Word2Vec or GloVe)
embeddings_path = "GoogleNews-vectors-negative300.bin"
embeddings_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_path, binary=True)

# Generate a vocabulary of words and their corresponding embeddings
vocab = {}
for sentence in sentences:
    for word in sentence.split():
        if word not in vocab and word in embeddings_model:
            vocab[word] = embeddings_model[word]

# Save the vocabulary to a file
vocab_path = "vocabulary.pkl"
with open(vocab_path, "wb") as f:
    pickle.dump(vocab, f)

In [62]:
print(len(vocab))

69705


In [65]:
from collections import Counter

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def pmi(word1, word2, sentences):
    pmi_scores = []

    for sentence_str in sentences:
        sentence = sentence_str.split()
        
        # Count the number of occurrences of each word in the sentence
        word_counts = Counter(sentence)

        # Check if word1 and word2 occur in the sentence
        if word1 not in word_counts or word2 not in word_counts:
            continue

        # Calculate the total number of words in the sentence
        total_words = len(sentence)

        # Calculate the number of times word1 and word2 co-occur in the sentence
        cooccurrences = sum(1 for i in range(len(sentence)-1) if word1 in sentence[i:] and word2 in sentence[i:])

        # Calculate the PMI between word1 and word2 for this sentence
        pmi_score = np.log2((cooccurrences * total_words) / (word_counts[word1] * word_counts[word2]))
        
        # Append the PMI score to the list of scores
        pmi_scores.append(pmi_score)

    # Calculate the average PMI score over all sentences
    if pmi_scores:
        return sum(pmi_scores) / len(pmi_scores)
    else:
        return 0.0
    


def generate_candidate_hypernyms(target_word, sentences, vocab, num_hypernyms=5, pmi_threshold=0.0):
    # Find all sentences that contain the target word
    target_sentences = [s for s in sentences if target_word in s.split()]

    # Find words that frequently co-occur with the target word
    cooccurring_words = set()
    for target_sentence in target_sentences:
        sentence_words = set(target_sentence.split())
        cooccurring_words.update(sentence_words)
    cooccurring_words.discard(target_word)


    # Calculate the PMI between the target word and each candidate hypernym
    pmi_scores = [(w, pmi(target_word, w, target_sentences)) for w in cooccurring_words]

    # Filter out candidate hypernyms with low PMI scores
    pmi_filtered = [(w, score) for (w, score) in pmi_scores if score > pmi_threshold]

    # Sort the candidate hypernyms by descending PMI score
    sorted_hypernyms = sorted(pmi_filtered, key=lambda x: x[1], reverse=True)

    # Return the top N candidate hypernyms
    top_hypernyms = [w for (w, score) in sorted_hypernyms[:num_hypernyms]]

    # Calculate the cosine similarity between the target word and each candidate hypernym
    embeddings = []
    for w in [target_word] + top_hypernyms:
        if w in vocab:
            embeddings.append(vocab[w])
    similarity_matrix = cosine_similarity(embeddings)
    similarities = similarity_matrix[0][1:]
    
    # Sort the candidate hypernyms by descending cosine similarity
    sorted_hypernyms = [w for _, w in sorted(zip(similarities, top_hypernyms), reverse=True)]
    
    return sorted_hypernyms

In [69]:
# Example usage
target_word = 'apple'

candidate_hypernyms = generate_candidate_hypernyms(target_word, sentences, vocab)
print(candidate_hypernyms)